In [ ]:
# Install rembg first
!pip install rembg pillow onnxruntime

import os
import io
from PIL import Image
from rembg import remove
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

dataset_dir = 'path'
output_dir = 'path'
os.makedirs(output_dir, exist_ok=True)

target_size = (3000, 3000)

def process_image(input_path, output_path):
    try:
        with Image.open(input_path).convert('RGBA') as img:
            img_bytes = io.BytesIO()
            img.save(img_bytes, format="PNG")
            img_bytes = img_bytes.getvalue()
            output_data = remove(img_bytes)
            result_image = Image.open(io.BytesIO(output_data)).convert('RGBA')
            white_bg = Image.new("RGBA", result_image.size, (255, 255, 255, 255))
            white_bg.paste(result_image, mask=result_image.split()[3])
            white_bg = white_bg.resize(target_size, Image.Resampling.LANCZOS)
            white_bg.convert("RGB").save(output_path, "JPEG")
        print(f" Processed: {os.path.basename(input_path)}")
    except Exception as e:
        print(f" Error processing {os.path.basename(input_path)}: {e}")

for root, _, files in os.walk(dataset_dir):
    for filename in files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(root, filename)
            rel_path = os.path.relpath(input_path, dataset_dir)
            save_dir = os.path.join(output_dir, os.path.dirname(rel_path))
            os.makedirs(save_dir, exist_ok=True)
            output_path = os.path.join(save_dir, f"{os.path.splitext(filename)[0]}.jpg")
            process_image(input_path, output_path)

print(" All images processed successfully!")
